In [8]:
import asyncio

import redis.asyncio as redis

STOPWORD = "STOP"


async def reader(channel: redis.client.PubSub):
    while True:
        message = await channel.get_message(ignore_subscribe_messages=True, timeout=None)
        if message is not None:
            print(f"(Reader) Message Received: {message}")
            if message["data"] == STOPWORD:
                print("(Reader) STOP")
                break


r = redis.from_url("redis://localhost?decode_responses=True")
async with r.pubsub() as pubsub:
    await pubsub.subscribe("channel:1", "channel:2")

    future = asyncio.create_task(reader(pubsub))

    await r.publish("channel:1", "Hello2")
    await r.publish("channel:2", "World")
    await r.publish("channel:1", STOPWORD)

    await future

(Reader) Message Received: {'type': 'message', 'pattern': None, 'channel': 'channel:1', 'data': 'Hello2'}
(Reader) Message Received: {'type': 'message', 'pattern': None, 'channel': 'channel:2', 'data': 'World'}
(Reader) Message Received: {'type': 'message', 'pattern': None, 'channel': 'channel:1', 'data': 'STOP'}
(Reader) STOP
